# 練習問題C 犯罪発生地域の特徴分析
【課題内容】<br>
「平成27年　区市町村の町丁別、罪種別及び手口別認知件数」をもとに、犯罪が多い地域とその特徴を分析する。<br>
それを踏まえて、実際に今後、犯罪件数の推計を進める計画を立て説明する。

# データの加工

まず下準備としてExcel上で全ての町丁を同じシート上にまとめ、'H27_data.xlsx'として保存しておく。

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
df= pd.read_excel('H27_data.xlsx', encoding='cp932')
#左から2, 3列目がインデックスとして読み込まれているので修正する。
df_r= df.reset_index()
df_i= df_r.set_index('level_0')
df_i.head(1)

,level_1,level_2,強盗,その他,Unnamed: 2,凶器準備集合,暴行,傷害,脅迫,恐喝,Unnamed: 8,金庫破り,学校荒し,事務所荒し,出店荒し,空き巣,忍込み,居空き,その他.1,Unnamed: 17,自動車盗,オートバイ盗,自転車盗,車上ねらい,自販機ねらい,工事場ねらい,すり,ひったくり,置引き,万引き,その他.2,Unnamed: 29,詐欺,占有離脱物横領,その他知能犯,賭博,その他刑法犯
level_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
千代田区丸の内１丁目,570,1,1,0,49,0,34,12,2,1,3,0,0,0,3,0,0,0,0,454,0,1,18,0,1,2,71,0,87,85,189,63,15,18,6,0,24


In [2]:
# カラム名を取得する
df_i.columns = ['総合計', '凶悪犯計', '強盗', '凶悪犯その他', '粗暴犯計', '凶器準備集合', '暴行', '傷害', '脅迫', '恐喝', '侵入窃盗計', '金庫破り', '学校荒し', '事務所荒し', '出店荒し', '空き巣', '忍込み', '居空き', '侵入窃盗その他', '非侵入窃盗計', '自動車盗', 'オートバイ盗','自転車盗', '車上ねらい', '自販機ねらい', '工事場ねらい', 'すり', 'ひったくり', '置引き', '万引き', '非侵入窃盗その他', 'その他計', '詐欺', '占有離脱物横領', 'その他知能犯', '賭博', 'その他刑法犯']
df_i.head(1)

,総合計,凶悪犯計,強盗,凶悪犯その他,粗暴犯計,凶器準備集合,暴行,傷害,脅迫,恐喝,侵入窃盗計,金庫破り,学校荒し,事務所荒し,出店荒し,空き巣,忍込み,居空き,侵入窃盗その他,非侵入窃盗計,自動車盗,オートバイ盗,自転車盗,車上ねらい,自販機ねらい,工事場ねらい,すり,ひったくり,置引き,万引き,非侵入窃盗その他,その他計,詐欺,占有離脱物横領,その他知能犯,賭博,その他刑法犯
level_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
千代田区丸の内１丁目,570,1,1,0,49,0,34,12,2,1,3,0,0,0,3,0,0,0,0,454,0,1,18,0,1,2,71,0,87,85,189,63,15,18,6,0,24


# 大分類別のデータの概観

In [3]:
# まずは犯罪件数が多い地域でどのような犯罪の占める割合が多いか大分類別にみてみる
df_sum= df_i[['総合計','凶悪犯計','粗暴犯計','侵入窃盗計','非侵入窃盗計','その他計']]
df_sum.sort_values(by='総合計', ascending=False).head(10)

,総合計,凶悪犯計,粗暴犯計,侵入窃盗計,非侵入窃盗計,その他計
level_0,,,,,,
新宿区新宿３丁目,1263,7,83,57,987,129
新宿区歌舞伎町１丁目,862,21,172,11,473,185
渋谷区宇田川町,838,3,77,7,575,176
新宿区西新宿１丁目,786,2,48,10,626,100
豊島区南池袋１丁目,780,2,72,5,599,102
豊島区東池袋１丁目,665,3,32,15,493,122
豊島区西池袋１丁目,579,0,64,2,416,97
千代田区丸の内１丁目,570,1,49,3,454,63
大田区蒲田５丁目,493,2,43,6,378,64


商業施設や飲食街のある地域で犯罪発生件数が多そう<br>
例えば、件数が多い地域として新宿区新宿3丁目、新宿区歌舞伎町1丁目、渋谷区宇田川町などが挙げられる<br>
一方で千代田区丸の内１丁目などターミナル駅のある地域の件数の多さも目立つ

In [4]:
# 大分類別の総合計に占める割合（%）を求める
cols = ['総合計', '凶悪犯','粗暴犯','侵入窃盗','非侵入窃盗','その他']
df_per = pd.DataFrame(index=[], columns=cols)
df_per['総合計']= df_sum['総合計']
df_per['凶悪犯']=df_sum['凶悪犯計']*100/df_sum['総合計']
df_per['粗暴犯']=df_sum['粗暴犯計']*100/df_sum['総合計']
df_per['侵入窃盗']=df_sum['侵入窃盗計']*100/df_sum['総合計']
df_per['非侵入窃盗']=df_sum['非侵入窃盗計']*100/df_sum['総合計']
df_per['その他']=df_sum['その他計']*100/df_sum['総合計']
df_per.round(1).sort_values(by='総合計', ascending=False).head(10)

,総合計,凶悪犯,粗暴犯,侵入窃盗,非侵入窃盗,その他
level_0,,,,,,
新宿区新宿３丁目,1263,0.6,6.6,4.5,78.1,10.2
新宿区歌舞伎町１丁目,862,2.4,20.0,1.3,54.9,21.5
渋谷区宇田川町,838,0.4,9.2,0.8,68.6,21.0
新宿区西新宿１丁目,786,0.3,6.1,1.3,79.6,12.7
豊島区南池袋１丁目,780,0.3,9.2,0.6,76.8,13.1
豊島区東池袋１丁目,665,0.5,4.8,2.3,74.1,18.3
豊島区西池袋１丁目,579,0.0,11.1,0.3,71.8,16.8
千代田区丸の内１丁目,570,0.2,8.6,0.5,79.6,11.1
大田区蒲田５丁目,493,0.4,8.7,1.2,76.7,13.0


多くの地域で非侵入窃盗が70%前後を占めている。<br>
一方で新宿区歌舞伎町1丁目や渋谷区道玄坂2丁目、豊島区西池袋１丁目などでは粗暴犯の割合が10%を超えている。<br>
これらの地域はともに繁華街があるために、暴行や恐喝などのトラブルが起きやすいと説明できそう。<br>
非侵入窃盗の内訳はもう少し細分化して見ていく必要があると考えられる。

### 粗暴犯が多い地域の特徴

In [5]:
df_sum.round(1).sort_values(by='粗暴犯計', ascending=False).head(10)

,総合計,凶悪犯計,粗暴犯計,侵入窃盗計,非侵入窃盗計,その他計
level_0,,,,,,
新宿区歌舞伎町１丁目,862,21,172,11,473,185
新宿区歌舞伎町２丁目,478,8,105,12,208,145
港区六本木３丁目,308,3,97,10,118,80
墨田区江東橋３丁目,345,3,84,6,197,55
新宿区新宿３丁目,1263,7,83,57,987,129
渋谷区宇田川町,838,3,77,7,575,176
豊島区南池袋１丁目,780,2,72,5,599,102
渋谷区道玄坂２丁目,492,9,65,1,327,90
豊島区西池袋１丁目,579,0,64,2,416,97


新宿区歌舞伎町１丁目、新宿区歌舞伎町２丁目、港区六本木３丁目など繁華街のある地域で粗暴犯の件数が多いことが確認できる。

# 非侵入窃盗の内訳

In [6]:
# 非侵入窃盗の小分類別発生件数
df_detail= df_i[['総合計','非侵入窃盗計', '自動車盗', 'オートバイ盗','自転車盗', '車上ねらい', '自販機ねらい', '工事場ねらい', 'すり', 'ひったくり', '置引き', '万引き', '非侵入窃盗その他']]
df_detail.sort_values(by='非侵入窃盗計', ascending=False).head(10)

,総合計,非侵入窃盗計,自動車盗,オートバイ盗,自転車盗,車上ねらい,自販機ねらい,工事場ねらい,すり,ひったくり,置引き,万引き,非侵入窃盗その他
level_0,,,,,,,,,,,,,
新宿区新宿３丁目,1263,987,0,0,82,5,6,0,97,4,56,519,218
新宿区西新宿１丁目,786,626,0,1,82,6,0,1,34,1,26,341,134
豊島区南池袋１丁目,780,599,0,0,34,4,2,0,57,0,28,223,251
渋谷区宇田川町,838,575,0,1,41,1,2,0,54,0,23,221,232
豊島区東池袋１丁目,665,493,0,1,63,3,7,0,57,2,72,79,209
新宿区歌舞伎町１丁目,862,473,1,3,73,2,3,0,22,2,32,79,256
千代田区丸の内１丁目,570,454,0,1,18,0,1,2,71,0,87,85,189
豊島区西池袋１丁目,579,416,0,0,59,5,2,0,16,0,15,156,163
大田区蒲田５丁目,493,378,0,2,254,5,0,3,8,1,15,32,58


非侵入窃盗のうち小分類別で見ると、万引きの件数が非常に大きい。<br>
あと小分類別で件数が多いのは自転車盗、すり、置引きあたり（非侵入窃盗その他を除く）<br>
非侵入窃盗全体に占めるこれらの小分類別の割合を見てみる

In [7]:
#非侵入窃盗の主要な小分類別割合
cols_1 = ['万引き', '自転車盗','すり','置引き']
df_detail_per = pd.DataFrame(index=[], columns=cols_1)
df_detail_per['非侵入窃盗計']= df_detail['非侵入窃盗計']
df_detail_per['万引き']=df_detail['万引き']*100/df_detail['非侵入窃盗計']
df_detail_per['自転車盗']=df_detail['自転車盗']*100/df_detail['非侵入窃盗計']
df_detail_per['すり']=df_detail['すり']*100/df_detail['非侵入窃盗計']
df_detail_per['置引き']=df_detail['置引き']*100/df_detail['非侵入窃盗計']
df_detail_per.round(1).sort_values(by='非侵入窃盗計', ascending=False).head(10)

,万引き,自転車盗,すり,置引き,非侵入窃盗計
level_0,,,,,
新宿区新宿３丁目,52.6,8.3,9.8,5.7,987
新宿区西新宿１丁目,54.5,13.1,5.4,4.2,626
豊島区南池袋１丁目,37.2,5.7,9.5,4.7,599
渋谷区宇田川町,38.4,7.1,9.4,4.0,575
豊島区東池袋１丁目,16.0,12.8,11.6,14.6,493
新宿区歌舞伎町１丁目,16.7,15.4,4.7,6.8,473
千代田区丸の内１丁目,18.7,4.0,15.6,19.2,454
豊島区西池袋１丁目,37.5,14.2,3.8,3.6,416
大田区蒲田５丁目,8.5,67.2,2.1,4.0,378


商業施設が集まる地域（新宿区新宿3丁目、新宿区西新宿１丁目など）では、万引きの割合が高いといえる。<br>
一方で、大田区蒲田５丁目、大田区西蒲田７丁目（いずれも蒲田駅前）では自転車盗の割合が異様に高い。<br>
万引きの発生件数順、自転車盗の発生件数順に並べかえて、それぞれの犯罪ごとに共通する要素がないか検討してみる。

### 万引きが多い地域の特徴

In [8]:
# 万引きの発生件数順に並べ替える
df_detail.round(1).sort_values(by='万引き', ascending=False).head(10)

,総合計,非侵入窃盗計,自動車盗,オートバイ盗,自転車盗,車上ねらい,自販機ねらい,工事場ねらい,すり,ひったくり,置引き,万引き,非侵入窃盗その他
level_0,,,,,,,,,,,,,
新宿区新宿３丁目,1263,987,0,0,82,5,6,0,97,4,56,519,218
新宿区西新宿１丁目,786,626,0,1,82,6,0,1,34,1,26,341,134
豊島区南池袋１丁目,780,599,0,0,34,4,2,0,57,0,28,223,251
渋谷区宇田川町,838,575,0,1,41,1,2,0,54,0,23,221,232
豊島区西池袋１丁目,579,416,0,0,59,5,2,0,16,0,15,156,163
渋谷区千駄ヶ谷５丁目,225,192,0,1,14,2,0,0,2,0,1,144,28
中央区銀座３丁目,180,160,0,0,7,1,0,1,2,0,4,113,32
千代田区有楽町２丁目,210,191,0,0,9,0,1,0,8,0,15,113,45
杉並区上荻１丁目,259,222,0,1,60,5,0,0,8,0,8,111,29


百貨店やショッピングセンターなどの商業施設が集積する地域で万引きの発生件数が多くなっていることが改めて確認できる

### 自転車盗の多い地域の特徴

In [9]:
#自転車盗の件数が多い順に並べかえる。
df_detail.round(1).sort_values(by='自転車盗', ascending=False).head(10)

,総合計,非侵入窃盗計,自動車盗,オートバイ盗,自転車盗,車上ねらい,自販機ねらい,工事場ねらい,すり,ひったくり,置引き,万引き,非侵入窃盗その他
level_0,,,,,,,,,,,,,
大田区蒲田５丁目,493,378,0,2,254,5,0,3,8,1,15,32,58
大田区西蒲田７丁目,467,376,0,1,212,8,0,0,3,3,20,74,55
北区赤羽１丁目,426,285,0,9,130,1,0,0,19,0,12,52,62
中野区中野５丁目,427,286,0,4,125,6,0,0,22,2,5,51,71
葛飾区新小岩１丁目,296,215,1,2,125,7,0,1,8,0,9,12,50
葛飾区亀有３丁目,289,236,0,5,113,18,1,0,2,2,5,59,31
大田区大森北１丁目,275,218,1,0,112,2,0,0,4,1,9,55,34
江戸川区西小岩１丁目,282,212,0,2,107,10,1,0,3,1,2,46,40
中野区中野４丁目,193,141,0,1,99,2,0,0,1,0,0,15,23


やはり蒲田駅周辺が突出して自転車盗の件数が多かった。<br>
一方でそれ以外の上位10地域でも総合計に占める自転車盗の割合は多いため、犯罪件数を推計するモデルを作る上で自転車盗が多い地域の共通の要素について考える必要はあるといえる。<br>
上位10地域を見ると、練馬区東大泉１丁目を除き全ての地域で山手線以外のJR線の駅が立地していることがわかる。なお、練馬区東大泉１丁目には西武池袋線の大泉学園駅が立地している。<br>
これは、山手線を除くJR線沿線（及び一部の私鉄駅）ではいずれも地域人口が多くかつ駅周辺に商店街などが集約しているため、地域住民が買い物時に自転車を駅周辺に駐車する頻度が高いなどの理由が考えられる。

# まとめ

以上より、犯罪が多い地域とその特徴は以下のようにまとめられる
- **商業施設の集積地**<br>
    - 新宿区新宿３丁目、新宿区西新宿１丁目、豊島区南池袋１丁目など<br>
    - これらの地域では、**万引き**の発生件数が多い
- **繁華街のある地域**<br>
    - 新宿区歌舞伎町１丁目、新宿区歌舞伎町２丁目、港区六本木３丁目など<br>
    - これらの地域では、**粗暴犯**の発生件数が多い<br>
- **山手線外のJR駅（及び一部の私鉄駅）周辺**<br>
    - 蒲田駅、赤羽駅、中野駅、新小岩駅周辺など<br>
    - これらの地域では**自転車盗**の発生件数が多い

# 犯罪件数推計の計画

犯罪件数の推計を以下の手順で進めていく。<br>
**1. 学習用データ・予測用データの生成**<br>
まず、3119の町丁のうち半分を学習用のデータとして、残り半分を予測用データとしてランダムに分類する。<br>
この予測用データについて犯罪件数を予測したい。<br>

**2. 変数選択**<br>
重回帰分析による予測モデルの推定を行う。<br>
まずは、これまでの考察をもとに説明変数に取り込むべき変数を選択する。<br>
町丁別の犯罪件数を非説明変数としたとき、上記より考えられる説明変数としては以下が挙げられると考えた。<br>

- **町丁別商業施設の年間商品販売額**
    - 商業施設が集積している地域ほど、万引き発生件数が多くなるという仮説が立てられる。
    - 個人商店も1とカウントする店舗数よりも、商品販売額の総計の方が説明変数として当てはまりがよいと考えられる
- **町丁別の飲食店舗数**
    - 繁華街があると、粗暴犯の件数が多くなるという仮説が立てられる。
    - 繁華街では小規模な飲食店が密集している傾向にあると考え、ここでは飲食店舗数を説明変数においた
    - 歌舞伎町や六本木で粗暴犯の件数が多いことから、データさえあればホストクラブの店舗数などの方が説明力がある可能性が考えられる。
- **その町丁における山手線外のJR駅の有無を表すダミー変数**
    - 地域内に山手線圏外のJR駅があると、自転車盗が多くなるという仮説が立てられる。
    - 地域人口の多さ・日用品店の駅前への集積・主たる移動手段が自転車である、といったJR線の駅に共通してみられる特徴が自転車盗が多い要因として考えられる
    - JR線以外でも都心と郊外をつなぐ私鉄の急行停車駅でも同様の傾向が見られる可能性があるため吟味が必要
    - JR駅周辺で特に顕著に自転車盗が見られていたことから、ここではまずJR駅の有無のダミー変数について考える

**3. 回帰モデルの作成**<br>
これを用いて町丁$i$の犯罪件数について以下のような回帰モデルを作ることができる。<br>
$$crime_i = \beta_0 + \beta_1shopsales_i + \beta_2restraunt_i + \gamma_0JRstation_i + u_i$$<br>
$shopsales_i$：町丁$i$における商業施設の年間商品販売額<br>
$restraunt_i$：町丁$i$における飲食店舗数<br>
$JRstation_i$：町丁$i$における山手線外のJR駅の有無を表すダミー変数<br>
$u_i$：誤差項

**4. 多重共線性のチェック**<br>
町丁$i$における商業施設の年間商品販売額と町丁$i$における飲食店舗数の間には正の相関があると考えられる。<br>
この二つの変数のVIF(Variance Inflation Factor)をチェックして、多重共線性が認められたらどちらかの変数を削除する。<br>
この場合、粗暴犯より万引きの発生件数の方が多いことから、町丁$i$における商業施設の年間商品販売額の方を説明変数として残すこととする。<br>

**5. 回帰モデルの推定**<br>
町丁$i$の犯罪件数の回帰モデルを推定する。<br>
それぞれの回帰係数が有意で、決定係数が高ければ、この回帰モデルを用いて残りの予測用データを用いた犯罪件数の予測を行えばよい。<br>
この回帰モデルの説明力が低い場合には、別の予測モデルを新たに作る必要がある。
